In [1]:
import gzip
import random
import json
import scipy
import numpy as np
import tensorflow as tf
from collections import defaultdict
import os
tf.compat.v1.enable_eager_execution()

# Data.py

Storing User, Item, Time tuples in `interactionTrain`

In [10]:
# put the "dataset" folder in the root directory
base = 'dataset'
# if base not in os.listdir('./'):
#     print(os.curdir)
#     print("dataset folder not found.")
#     # break # made available in the final py file
# else:
interactions_Jewelry_train = os.path.join(base, 'interactions_Jewelry_train.json')
interactions_Jewelry_train_aux = os.path.join(base, 'interactions_Jewelry_train_aux.json')
interactions_Jewelry_train_record = os.path.join(base, 'interactions_Jewelry_train_record_aux.json')
interactions_Jewelry_train_time = os.path.join(base, 'interactions_Jewelry_train_time_aux.json')
interactions_Jewelry_validate = os.path.join(base, 'interactions_Jewelry_validate.json')
interactions_Jewelry_test = os.path.join(base, 'interactions_Jewelry_test.json')
CNN_AES = "CNN_AES_feature.txt"
id2num_dict = "id2num_dict_Jewelry.json"
if CNN_AES not in os.listdir(os.path.join('.', base, "features")) or id2num_dict not in os.listdir(os.path.join('.', base, "id2num_dict")):
    print("CNN_AES and id2num_dict folders not found.")
    # break
else:
    CNN_AES = os.path.join(base, "features", CNN_AES)
    id2num_dict = os.path.join(base, "id2num_dict", id2num_dict)

In [11]:
userIDs = set()
itemIDs = set()
#interactionsTrain = []
user_to_item = {}
user_time_to_item = {}
time_to_item = {}

with open(interactions_Jewelry_train) as json_file:
    data = json.load(json_file)

In [12]:
for d in data:
    u = d[0]
    i = d[1]
    r = d[2]
    #interactionsTrain.append((u,i,r))
    userIDs.add(u)
    itemIDs.add(i)
    if u in user_to_item:
        user_to_item[u].add(i)
    else:
        user_to_item[u] = {i}
    if (u,r) in user_time_to_item:
        user_time_to_item[(u,r)].add(i)
    else:
        user_time_to_item[(u,r)] = {i}
    if r in time_to_item:
        time_to_item[r].add(i)
    else:
        time_to_item[r] = {i}


Storing CNN-AES Feature in `cnn`

In [13]:
with open(CNN_AES) as cnn_txt:
    cnn = cnn_txt.readlines()

Read id2num_dict in which contains item id and the dictionary index

In [14]:
with open(id2num_dict) as id2num_dict_json:
    id2num = id2num_dict_json.readlines()

*From Library.py*
Functions for:
- Model Evaluation
- Data Reading

In [24]:
import numpy as np
from numpy import *
from xlrd import open_workbook
from xlutils.copy import copy
import json

"""
Evaluation Metrics
"""

def evaluation_F1(order, top_k, positive_item):
    e = 0.00000000000001
    top_k_items = set(order[0: top_k])
    positive_item = set(positive_item)
    precision = len(top_k_items & positive_item) / (len(top_k_items) + e)
    recall = len(top_k_items & positive_item) / (len(positive_item) + e)
    F1 = 2 * precision * recall / (precision + recall + e)
    return F1

def evaluation_NDCG(order, top_k, positive_item):
    top_k_item = order[0: top_k]
    e = 0.0000000001
    Z_u = 0
    temp = 0
    for i in range(0, top_k):
        Z_u += 1 / log2(i + 2)
        if top_k_item[i] in positive_item:
            temp += 1 / log2(i + 2)
    NDCG = temp / (Z_u + e)
    return NDCG

def save_result(intro, F1, NDCG, path):
    rexcel = open_workbook(path)
    rows = rexcel.sheets()[0].nrows
    excel = copy(rexcel)
    table = excel.get_sheet(0)
    row = rows
    table.write(row, 0, intro)
    #table.write(row, 2, 'F1')
    for i in range(len(F1)):
        table.write(row, i + 3, F1[i])
    #table.write(row, len(F1) + 4, 'NDCG')
    for i in range(len(NDCG)):
        table.write(row, i + len(F1) + 5, NDCG[i])
    excel.save(path)

"""
Read Data
"""
def readdata(dataset):
    #file paths
    path_train = interactions_Jewelry_train
    path_train_aux = interactions_Jewelry_train_aux
    path_validate = interactions_Jewelry_validate
    path_test = interactions_Jewelry_test
    # read files
    with open(path_train) as f:
        line = f.readline()
        train_data = json.loads(line)
    f.close()
    P = 0
    Q = 0
    for [u, i, r] in train_data:
        if u > P:
            P = u
        if i > Q:
            Q = i
    with open(path_train_aux) as f:
        line = f.readline()
        train_data_aux = json.loads(line)
    f.close()
    with open(path_validate) as f:
        line = f.readline()
        validate_data = json.loads(line)
    f.close()
    with open(path_test) as f:
        line = f.readline()
        test_data = json.loads(line)
    f.close()
    return train_data, train_data_aux, validate_data, test_data, P + 1, Q + 1 # P: last user_id, last item_id

def readdata_time(dataset):
    #file paths
    path_train_record_aux = interactions_Jewelry_train_record
    path_train_time_aux = interactions_Jewelry_train_time
    # read files
    with open(path_train_record_aux) as f:
        line = f.readline()
        train_record_aux = json.loads(line)
    f.close()
    with open(path_train_time_aux) as f:
        line = f.readline()
        train_time_aux = json.loads(line)
    f.close()
    return train_record_aux, train_time_aux, len(train_time_aux)

def read_feature(feature, dataset, Q):
    path_feature = CNN_AES
    path_dict = id2num_dict
    with open(path_dict) as f:
        line = f.readline()
        item_i2num_dict = json.loads(line)
    f.close()
    f = open(path_feature, 'r')
    line = eval(f.readline())
    feature = line[1]
    K = len(feature)
    F = np.zeros((Q, K))
    for i in range(0, Q):
        F[i] = feature
    for line in f:
        line = eval(line)
        item_id = line[0]
        feature = line[1]
        try:
            item_num = item_i2num_dict[item_id]
            F[item_num] = feature
        except:
            continue
    return F

def get_feature(dataset, Q):
    # to load features
    feat = [3]
    feat_list = ['CNN', 'AES', 'CH', 'CNN_AES']             # feature list
    F = read_feature(feat_list[feat[0]], dataset, Q)
    for i in range(1, len(feat)):
        F = np.hstack((F, read_feature(feat_list[feat[i]], dataset, Q)))
    return F

# Model.py

In [25]:

class DCFA(tf.keras.Model):
    def __init__(self, P, Q, R, I, J, F, reg=1.5, mom=0.1):
        """
        Initialization of the Visually_based Bayesian Personalized Ranking Model with Aesthetic Features

        :param P: Number of Users
        :param Q: Number of Items
        :param R: Number of Time Intervals
        :param I: Dimension of each Latent Items
        :param J: Dimension of each Latent Items
        :param F: The CNN-AES Feature Matrix
        :param reg: regularize lambda
        :param mom: momentum gamma
        """
        super(DCFA, self).__init__()
        # Latent Items
        self.U = tf.Variable(np.array([np.array([(random.random() / math.sqrt(I)) for j in range(I)]) for i in range(P)]))
        print("Dimension of U: ", self.U.shape)
        self.V = tf.Variable(np.array([np.array([(random.random() / math.sqrt(I)) for j in range(I)]) for i in range(Q)]))
        print("Dimension of V: ", self.V.shape)
        self.W = tf.Variable(np.array([np.array([(random.random() / math.sqrt(J)) for j in range(J)]) for i in range(Q)]))
        print("Dimension of W: ", self.W.shape)
        self.T = tf.Variable(np.array([np.array([(random.random() / math.sqrt(J)) for j in range(J)]) for i in range(R)]))
        print("Dimension of T: ", self.T.shape)
        # Extract CNN-AES Features
        self.F, self.K = F, len(F[0])
        print("Dimension of F: ", self.F.shape)
        print("Size of F: ", self.K)
        self.M = tf.Variable(np.array([np.array([(random.random() / math.sqrt(self.K)) for j in range(self.K)]) for i in range(P)]))
        print("Dimension of M: ", self.M.shape)
        self.N = tf.Variable(np.array([np.array([(random.random() / math.sqrt(self.K)) for j in range(self.K)]) for i in range(R)]))
        print("Dimension of N: ", self.N.shape)
        # regularize coefficient and momentum coefficient
        self.reg = reg
        self.mom = mom
        # self.top_k = [5, 10, 20, 50, 100]

    def score(self, u, v, r):
        """
        Given a (user, item, time) tuple, return the score associated with the relevancy of the given item to given user
        at given timestamp

        :param u: user_id
        :param v: item_id
        :param r: timestamp
        :return: BPR score for the (user, item, time) tuple
        """
        # B = (tf.expand_dims(self.U[u], axis=0) @ tf.transpose(self.V) + tf.expand_dims(self.M[u], axis=0) @ tf.transpose(self.F))[0][v]
        # C = (tf.expand_dims(self.T[r], axis=0) @ tf.transpose(self.W) + tf.expand_dims(self.N[r], axis=0) @ tf.transpose(self.F))[0][v]
        #print(B.shape, C.shape)
        #return tf.tensordot(B, C, axes=0), B, C
        # print("score" + str(B*C))
        # return B*C

        B = tf.expand_dims(self.U[u], axis=0) @ tf.transpose(self.V) + tf.expand_dims(self.M[u], axis=0) @ tf.transpose(self.F)
        C = tf.expand_dims(self.T[r], axis=0) @ tf.transpose(self.W) + tf.expand_dims(self.N[r], axis=0) @ tf.transpose(self.F)
        # print("B Shape: ", B.shape)
        # print("C Shape: ", C.shape)
        # print("A Shape: ", (tf.squeeze(B)*tf.squeeze(C)).shape)
        return (tf.squeeze(B)*tf.squeeze(C))[v], tf.squeeze(B)[v], tf.squeeze(C)[v]

    def score_batch(self, u, r):
        """
        Given a (user, time) pair, return an array of scores associated with the given user and given time
        :param u:
        :param r:
        :return:
        """
        B = tf.expand_dims(self.U[u], axis=0) @ tf.transpose(self.V) + tf.expand_dims(self.M[u], axis=0) @ tf.transpose(self.F)
        C = tf.expand_dims(self.T[r], axis=0) @ tf.transpose(self.W) + tf.expand_dims(self.N[r], axis=0) @ tf.transpose(self.F)
        return tf.squeeze(B)*tf.squeeze(C), tf.squeeze(B), tf.squeeze(C)

    def call(self, u, v, v_prime, r):
        """
        Calculate the BPR_OPT distance, which is a metric that measures the scoring distance between positive sample
        and the negative sample of the given (user, item, time) tuple

        :param u: user_id
        :param v: item_id
        :param v_prime: negative sample of item_id
        :param r: timestamp
        :return: loss value for the given tuple
        """
        A_i, B_i, C_i = self.score(u, v, r)
        A_j, B_j, C_j = self.score(u, v_prime, r)

        # print("B_ij: ", B_i-B_j)
        # print("C_ij: ", C_i-C_j)

        A_loss = tf.math.log(tf.keras.activations.sigmoid(A_i-A_j))
        B_loss = tf.math.log(tf.keras.activations.sigmoid(B_i-B_j))
        C_loss = tf.math.log(tf.keras.activations.sigmoid(C_i-C_j))
        # B_loss = tf.keras.activations.sigmoid(B_i-B_j)
        # C_loss = tf.keras.activations.sigmoid(C_i-C_j)

        # print("A loss: ", A_loss)
        # print("B loss: ", B_loss)
        # print("C loss: ", C_loss)

        return tf.convert_to_tensor(A_loss + self.mom * B_loss + self.mom * C_loss)
        # TODO: Fix Lambda regularization term (i.e. self.reg and self.mom)

    def reg(self):
        """
        Return the regularization value for the current latent terms

        :return: regularization term
        """
        return self.lamb * (tf.reduce_sum(self.betaU**2) + tf.reduce_sum(self.betaI**2) +
                            tf.reduce_sum(self.gammaU**2) + tf.reduce_sum(self.gammaI**2))

# Engine.py

In [28]:
class engine():
    def __init__(self, learning_rate=1e-3, batch_size=64, k=5):
        """

        :param n_size:
        """

        self.optimizer = None
        self.model = None

        # define hyperparameter
        self.k = k
        self.batch_size = batch_size
        self.lr = learning_rate

        # save data
        self.userIDs = set()
        self.itemIDs = set()
        self.user_to_item = {}
        self.user_time_to_item = {}
        self.time_to_item = {}

        # self.train, self.test
        self.cnn = None
        self.id2num = None
        self.train_data = None
        self.train_data_aux = None
        self.validate_data = None
        self.test_data = None
        self.train_record_aux = None
        self.train_time_aux = None
        self.R = None
        self.P = None
        self.Q = None
        self.F = None


    def run(self):
        self.read_data()
        self.read_feature_data()
        self.create_model()


    def train_batch(self):
        pass


    def test_batch(self):
        pass



    def read_data(self):
        # put the "dataset" folder in the root directory
        base = 'dataset'
        if base not in os.listdir('./'):
            print(os.curdir)
            print("dataset folder not found.")
            # break # made available in the final py file
        else:
            interactions_Jewelry_train = os.path.join(base, 'interactions_Jewelry_train.json')
            interactions_Jewelry_train_aux = os.path.join(base, 'interactions_Jewelry_train_aux.json')
            interactions_Jewelry_train_record = os.path.join(base, 'interactions_Jewelry_train_record_aux.json')
            interactions_Jewelry_train_time = os.path.join(base, 'interactions_Jewelry_train_time_aux.json')
            interactions_Jewelry_validate = os.path.join(base, 'interactions_Jewelry_validate.json')
            interactions_Jewelry_test = os.path.join(base, 'interactions_Jewelry_test.json')
            CNN_AES = "CNN_AES_feature.txt"
            id2num_dict = "id2num_dict_Jewelry.json"
        if CNN_AES not in os.listdir(os.path.join('.', base, "features")) or id2num_dict not in os.listdir(os.path.join('.', base, "id2num_dict")):
            print("CNN_AES and id2num_dict folders not found.")
            # break
        else:
            CNN_AES = os.path.join(base, "features", CNN_AES)
            id2num_dict = os.path.join(base, "id2num_dict", id2num_dict)

        #May need to add more
        self.read_interaction_data(interactions_Jewelry_train)


    def read_interaction_data(self, filePath, data_size=50):
        with open(filePath) as json_file:
            data = json.load(json_file)

        for d in data[:data_size]:
            u = d[0]
            i = d[1]
            r = d[2]
            #interactionsTrain.append((u,i,r))
            self.userIDs.add(u)
            self.itemIDs.add(i)
            if u in self.user_to_item:
                self.user_to_item[u].add(i)
            else:
                self.user_to_item[u] = {i}
            if (u,r) in self.user_time_to_item:
                self.user_time_to_item[(u,r)].add(i)
            else:
                self.user_time_to_item[(u,r)] = {i}
            if r in self.time_to_item:
                self.time_to_item[r].add(i)
            else:
                self.time_to_item[r] = {i}


    def read_feature_data(self):
        with open(CNN_AES) as cnn_txt:
            self.cnn = cnn_txt.readlines()
        with open(id2num_dict) as id2num_dict_json:
            self.id2num = id2num_dict_json.readlines()

        # setup
        feat = [3]                          # feature selecting, 0 for CNN, 1 for AES, 2 for CH, 3 for CNN+AES
        dataset = 5                         # Datasets selecting 0 to 5 for 'All', '_Women', '_Men', '_CLothes', '_Shoes', '_Jewelry' respectively
        dataset_list = ['', '_Women', '_Men', '_CLothes', '_Shoes', '_Jewelry']
        # load data
        self.train_data, self.train_data_aux, self.validate_data, self.test_data, self.P, self.Q = readdata(dataset_list[dataset])
        # load data for tensor factorization
        self.train_record_aux, self.train_time_aux, self.R = readdata_time(dataset_list[dataset])
        # load features
        self.F = get_feature(dataset_list[dataset], self.Q) # CNN_AES Features

    def create_model(self):
        self.model = DCFA(P=self.P, Q=self.Q, R=self.R, I=200, J=200, F=self.F, reg=1.5, mom=0.1)

    #def create_negative(self):

In [29]:
pipe = engine()

In [30]:
pipe.run()

Metal device set to: Apple M1 Pro


2022-03-10 18:36:21.853853: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-10 18:36:21.854651: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Dimension of U:  (15924, 200)
Dimension of V:  (3607, 200)
Dimension of W:  (3607, 200)
Dimension of T:  (238, 200)
Dimension of F:  (3607, 1746)
Size of F:  1746
Dimension of M:  (15924, 1746)
Dimension of N:  (238, 1746)


In [31]:
pipe.model.score(12308, 7, 182)

(<tf.Tensor: shape=(), dtype=float64, numpy=0.05654044189814102>,
 <tf.Tensor: shape=(), dtype=float64, numpy=0.23054644127853194>,
 <tf.Tensor: shape=(), dtype=float64, numpy=0.24524534659735803>)

In [32]:
pipe.model.call(12308, 0, 1, 182)

<tf.Tensor: shape=(), dtype=float64, numpy=-0.8334998114738152>

In [37]:
# Negative Sampling Function
from random import choice

def negative(u: int, itemIDs: set, user_items=user_to_item) -> int:
    """Maybe get the user's historical interation items"""
    historical_items = list(user_items[u])
    itemIDs_list = list(itemIDs) # POTENTIAL RUNTIME ISSUE
    negative_item = choice(itemIDs_list)
    while negative_item in historical_items:
        negative_item = choice(itemIDs_list)
    return negative_item


KeyError: 21

In [43]:
# Gradient Function

def grad(u, i, negative, r):
    with tf.GradientTape() as tape:
        loss_value = pipe.model.call(u, i, negative, r)
    # change user, item, time ids to variables
    return loss_value, tape.gradient(loss_value, pipe.model.trainable_variables)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
loss_value, gradients = grad(12308, 0, 1, 182)

print("Step: {}, Loss: {}".format(optimizer.iterations.numpy(),
                                  pipe.model.call(12308, 0, 1, 182).numpy()))

optimizer.apply_gradients(zip(gradients, pipe.model.trainable_variables))

print("Step: {}, Loss: {}".format(optimizer.iterations.numpy(),
                                  pipe.model.call(12308, 0, 1, 182).numpy()))

Step: 0, Loss: -0.8334998114738152


NotFoundError: No registered 'ResourceApplyGradientDescent' OpKernel for 'GPU' devices compatible with node {{node ResourceApplyGradientDescent}}
	 (OpKernel was found, but attributes didn't match) Requested Attributes: T=DT_DOUBLE, use_locking=true
	.  Registered:  device='GPU'; T in [DT_FLOAT]
  device='CPU'; T in [DT_HALF]
  device='CPU'; T in [DT_BFLOAT16]
  device='CPU'; T in [DT_FLOAT]
  device='CPU'; T in [DT_DOUBLE]
  device='CPU'; T in [DT_COMPLEX64]
  device='CPU'; T in [DT_COMPLEX128]
 [Op:ResourceApplyGradientDescent]

In [44]:
num_epochs = 10
loss_per_epoch = []
for epoch in range(1,num_epochs+1):
    loss_per_batch = []
    for u, i, r in data[:100]: # iterate over all interactions
        # create negative sample
        negative_item = negative(u, itemIDs, user_to_item)
        loss_value, gradients = grad(u, i, negative_item, r)
        optimizer.apply_gradients(zip(gradients, pipe.model.trainable_variables))
        loss_per_batch.append(loss_value)

        # print("Step: {}, Loss: {}".format(optimizer.iterations.numpy(),
        #                                   pipe.model.call(u, i, negative_item, r).numpy()))
    loss_per_epoch.append(sum(loss_per_batch)/len(loss_per_batch))
    print("Epoch: {}, Loss: {}".format(epoch, loss_per_batch[-1]))


NotFoundError: No registered 'ResourceApplyGradientDescent' OpKernel for 'GPU' devices compatible with node {{node ResourceApplyGradientDescent}}
	 (OpKernel was found, but attributes didn't match) Requested Attributes: T=DT_DOUBLE, use_locking=true
	.  Registered:  device='GPU'; T in [DT_FLOAT]
  device='CPU'; T in [DT_HALF]
  device='CPU'; T in [DT_BFLOAT16]
  device='CPU'; T in [DT_FLOAT]
  device='CPU'; T in [DT_DOUBLE]
  device='CPU'; T in [DT_COMPLEX64]
  device='CPU'; T in [DT_COMPLEX128]
 [Op:ResourceApplyGradientDescent]

In [167]:
negative_item

2490